# Pix2Pix

- 조건부 적대 네트워크를 사용한 이미지-이미지 변환에 설명 된대로 조건부 GAN을 사용하여 이미지를 이미지로 변환하는 방법

- 이 기술을 사용하여 흑백 사진에 색을 입히고 Google지도를 Google 어스로 변환하는 등의 작업을 수행 할 수 있음

- 아래의 사진처럼 건물의 외관을 실제 건물로 변환할 수 있음

  <img src="https://www.tensorflow.org/images/gan/pix2pix_1.png">

  <img src="https://www.tensorflow.org/images/gan/pix2pix_2.png">

In [ ]:
import tensorflow as tf
import os
import time

from matplotlib import pyplot as plt
from IPython import display

## 데이터셋 로드

- 임의 jittering에서 286x286 크기의 이미지가 무작위로 256x256으로 잘림

- 랜덤 미러링에서는 이미지가 좌우로 무작위로 반전됨

- https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz

In [ ]:
URL = 'https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz'
path_to_zip = tf.keras.utils.get_file('facades.tar.gz', origin=URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'facades/')

Exception: URL fetch failure on https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz: 404 -- Not Found

- 이미지를 [-1, 1] 사이의 값으로 정규화

- 아래 이미지에서 볼 수 있듯이 무작위 지 터링을 겪고 있음을 알 수 있음

  1. 더 큰 높이와 너비로 이미지 크기 조정

  2. 대상 크기로 무작위로 자르기

  3. 이미지를 가로로 무작위로 뒤집기

## 입력 파이프라인
- `tf.data.Dataset`을 이용하여 train, test dataset을 생성

## 생성기 구축

- 제너레이터의 아키텍처는 수정 된 U-Net입니다.

- 인코더의 각 블록은 (Conv-> Batchnorm-> Leaky ReLU)

- 디코더의 각 블록은 (Transposed Conv-> Batchnorm-> Dropout (처음 3 개 블록에 적용)-> ReLU)

- 인코더와 디코더 사이에는 스킵 연결이 있음(U-Net에서와 같이).

- Generator 손실

  - 생성 된 이미지와 이미지 배열의 시그모이드 교차 엔트로피 손실

  - 논문에는 생성 된 이미지와 대상 이미지 사이의 MAE (평균 절대 오차) 인 L1 손실도 포함

  - 이렇게하면 생성된 이미지가 대상 이미지와 구조적으로 유사 해짐

  - 총 generator loss을 계산하는 공식 = gan_loss + LAMBDA * l1_loss, 여기서 LAMBDA = 100.이 값은 논문 저자가 결정함

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/gen.png?raw=1">


## 판별자 구축

- 판별자는 PatchGAN

- 식별기의 각 블록은 (Conv-> BatchNorm-> Leaky ReLU)

- 마지막 레이어 이후의 출력 모양은 (batch_size, 30, 30, 1)

- 출력의 각 30x30 패치는 입력 이미지의 70x70 부분을 분류 (이러한 아키텍처를 PatchGAN이라고 함)

- 판별기는 2 개의 입력을 받음

  - 실제로 분류해야하는 입력 이미지와 대상 이미지

  - 입력 이미지와 생성된 이미지 (생성기의 출력)를 가짜로 분류

  - 이 두 입력을 코드에서 함께 연결 (`tf.concat([inp, tar], axis=-1)`).

## 판별자 손실

- 판별기 손실 함수는 2 개의 입력을 받음
  - 실제 이미지
  
  - 생성 된 이미지

- real_loss는 **실제 이미지**와 **이미지의 배열**의 시그모이드 교차 엔트로피 손실(**실제 이미지 이므로**)

- generated_loss는 **생성 된 이미지**와 **0의 배열**의 시그모이드 교차 엔트로피 손실(**가짜 이미지이기 때문에**)

- total_loss는 real_loss와 generated_loss의 합

- 판별자에 대한 학습 절차

  <img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/generative/images/dis.png?raw=1">
  

## 옵티마이저 설정

- 생성자, 판별자

  - Adam

  - $lr = 2e-4$
  
  - $beta_1 = 0.5$


- 체크포인트 설정

## 이미지 생성

- 훈련 중에 일부 이미지를 그리는 함수를 작성

  - 테스트 데이터 세트의 이미지를 생성기로 전달

  - 생성기는 입력 이미지를 출력으로 변환

  - 마지막 단계는 예측을 그리는 것

## 학습

- 학습 과정

  - 각 예제 입력에 대해 출력을 생성

  - 판별기는 input_image와 생성된 이미지를 첫 번째 입력으로 받음

  - 두 번째 입력은 input_image 및 target_image

  - 생성기와 판별기 손실을 계산
  
  - 그런 다음 생성기 및 판별 변수 (입력)에 대한 손실의 기울기를 계산하고이를 최적화기에 적용

  - 손실을 TensorBoard에 기록

- 실제 훈련 루프

  - Epoch 수를 반복

  - 각 epoch에서 디스플레이를 지우고 generate_images 를 실행 generate_images 진행 상황을 표시

  - 각 에포크에서 학습 데이터 세트를 반복하여 '.'를 출력
  - 20 epoch마다 체크 포인트를 저장

- 위에 표시되는 Tensorboard에서 새로고침을 누르면 그래프가 reload됨

- 학습 시간 매우 많이 소요

## 최신 checkpoints 복원 및 test

## 테스트 데이터셋을 사용하여 이미지 생성